In [1]:
!pip install torch megatron-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 743.6/743.6 kB 45.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 108.5 MB/s eta 0:00:00
  Created wheel for megatron-core: filename=megatron_core-0.13.1-cp312-cp312-linux_x86_64.whl size=2092872 sha256=636dd8d7f4cd1511940158225bfeb9efb7c091891cbb69bde3d52504c9c68e50
  Stored in directory: /root/.cache/pip/wheels/a2/89/3a/e73801dac99baaeb15755bd738fab1a9143b2dd48434aeacba
Successfully built megatron-core
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

In [5]:
%%writefile gpt_oss_trainer.py
# gpt_oss_trainer.py
import os
from pathlib import Path
from functools import partial
from typing import Dict, Iterable, Optional, Tuple, Callable

import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

from megatron.core import parallel_state
from megatron.core import dist_checkpointing
from megatron.core.pipeline_parallel.schedules import get_forward_backward_func
from megatron.core.tensor_parallel.random import model_parallel_cuda_manual_seed
from megatron.core.transformer.enums import AttnBackend
from megatron.core.transformer.transformer_config import TransformerConfig
from megatron.core.models.gpt.gpt_model import GPTModel
from megatron.core.models.gpt.gpt_layer_specs import get_gpt_layer_local_spec
from megatron.core.datasets.utils import compile_helpers
from megatron.core.datasets.blended_megatron_dataset_builder import BlendedMegatronDatasetBuilder
from megatron.core.datasets.gpt_dataset import GPTDatasetConfig, MockGPTDataset
from placeholder_tokenizer import _NullTokenizer

_SEQUENCE_LENGTH = 4096
_BASE_VOCAB_SIZE = 201_088


class DistributedEnvironment:
    @staticmethod
    def initialize(
        tensor_model_parallel_size: int = 1,
        pipeline_model_parallel_size: int = 1,
        expert_model_parallel_size: int = 1,
    ) -> None:

        parallel_state.destroy_model_parallel()


        rank = int(os.environ.get("LOCAL_RANK", "0"))
        world_size = torch.cuda.device_count()
        torch.cuda.set_device(rank)
        torch.distributed.init_process_group(
            backend="nccl", world_size=world_size, rank=rank
        )

        # Megatron parallel init
        parallel_state.initialize_model_parallel(
            tensor_model_parallel_size=tensor_model_parallel_size,
            pipeline_model_parallel_size=pipeline_model_parallel_size,
            expert_model_parallel_size=expert_model_parallel_size,
        )

    @staticmethod
    def device() -> torch.device:
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")



class GPT20BStyleFactory:

    @staticmethod
    def build(
        sequence_length: int = _SEQUENCE_LENGTH,
        vocab_size: int = _BASE_VOCAB_SIZE,
        hidden_size: int = 2880,
        num_layers: int = 24,
        num_attention_heads: int = 64,
        num_query_groups: int = 8,
        kv_channels: int = 48,
    ) -> GPTModel:
        tp_size = parallel_state.get_tensor_model_parallel_world_size()
        pp_size = parallel_state.get_pipeline_model_parallel_world_size()

        config = TransformerConfig(
            num_layers=num_layers,
            hidden_size=hidden_size,
            num_attention_heads=num_attention_heads,
            tensor_model_parallel_size=tp_size,
            pipeline_model_parallel_size=pp_size,
            num_query_groups=num_query_groups,
            add_bias_linear=False,
            normalization="RMSNorm",
            gated_linear_unit=True,
            activation_func=F.silu,
            num_moe_experts=32,
            moe_ffn_hidden_size=4 * hidden_size,
            moe_router_topk=4,
            moe_aux_loss_coeff=1e-2,
            attention_backend=AttnBackend.flash,
            apply_residual_connection_post_layernorm=False,
            use_cpu_initialization=True,
            pipeline_dtype=torch.bfloat16,
            kv_channels=kv_channels,
        )

        model = GPTModel(
            config=config,
            transformer_layer_spec=get_gpt_layer_local_spec(),
            vocab_size=vocab_size,
            max_sequence_length=sequence_length,
            position_embedding_type="rope",
        )
        return model


class GPTMockDataModule:

    def __init__(
        self,
        sequence_length: int = _SEQUENCE_LENGTH,
        vocab_size: int = _BASE_VOCAB_SIZE,
        batch_size: int = 1,
        seed: int = 0,
        shuffle: bool = True,
    ) -> None:
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.seed = seed
        self.shuffle = shuffle

    def setup(self) -> None:
        # Compile CUDA helpers once per job (rank 0 guards)
        if torch.distributed.is_available() and torch.distributed.is_initialized():
            if torch.distributed.get_rank() == 0:
                compile_helpers()
            torch.distributed.barrier()
        else:
            compile_helpers()

        cfg = GPTDatasetConfig(
            random_seed=self.seed,
            sequence_length=self.sequence_length,
            reset_position_ids=False,
            reset_attention_mask=False,
            eod_mask_loss=False,
            tokenizer=_NullTokenizer(vocab_size=self.vocab_size),
            mid_level_dataset_surplus=0.005,
        )

        datasets = BlendedMegatronDatasetBuilder(
            MockGPTDataset, [1000, None, None], lambda: True, cfg
        ).build()
        self._train_ds = datasets[0]

    def train_dataloader(self) -> Iterable[Dict[str, torch.Tensor]]:
        return DataLoader(self._train_ds, batch_size=self.batch_size, shuffle=self.shuffle)



class CausalLMForwardStep:

    def __init__(self, device: torch.device):
        self.device = device

    def __call__(
        self, data_iterator: Iterable[Dict[str, torch.Tensor]], model: torch.nn.Module
    ) -> Tuple[torch.Tensor, Callable]:
        def loss_func(loss_mask: torch.Tensor, output_tensor: torch.Tensor):
            losses = output_tensor.float()
            loss_mask_f = loss_mask.view(-1).float()
            loss = torch.sum(losses.view(-1) * loss_mask_f) / loss_mask_f.sum()
            return loss, {"lm loss": loss}

        batch = next(data_iterator)
        tokens = batch["tokens"].to(self.device)
        attention_mask = batch["attention_mask"].to(self.device)
        position_ids = batch["position_ids"].to(self.device)
        labels = batch["labels"].to(self.device)
        loss_mask = batch["loss_mask"].to(self.device)

        output_tensor = model(tokens, position_ids, attention_mask, labels=labels)
        return output_tensor, partial(loss_func, loss_mask)


class CheckpointManager:
    @staticmethod
    def save(model: GPTModel, path: str) -> None:
        Path(path).mkdir(parents=True, exist_ok=True)
        sharded_sd = model.sharded_state_dict(prefix="")
        dist_checkpointing.save(sharded_state_dict=sharded_sd, checkpoint_dir=path)

    @staticmethod
    def load(model: GPTModel, path: str) -> GPTModel:
        sharded_sd = model.sharded_state_dict(prefix="")
        ckpt = dist_checkpointing.load(sharded_state_dict=sharded_sd, checkpoint_dir=path)
        model.load_state_dict(ckpt)
        return model


class MegatronTrainer:

    def __init__(
        self,
        model: GPTModel,
        optimizer: torch.optim.Optimizer,
        device: Optional[torch.device] = None,
        num_microbatches: int = 1,
        micro_batch_size: int = 2,
        seq_length: int = _SEQUENCE_LENGTH,
        decoder_seq_length: Optional[int] = None,
        forward_only: bool = False,
        log_interval: int = 1,
    ) -> None:
        self.model = model
        self.optimizer = optimizer
        self.device = device or DistributedEnvironment.device()
        self.num_microbatches = num_microbatches
        self.micro_batch_size = micro_batch_size
        self.seq_length = seq_length
        self.decoder_seq_length = decoder_seq_length or seq_length
        self.forward_only = forward_only
        self.log_interval = log_interval

        self.forward_backward = get_forward_backward_func()
        self.forward_step = CausalLMForwardStep(self.device)

        # Place model on device (important for non-pipeline stages)
        self.model.to(self.device)

    def training_step(self, data_iter: Iterable[Dict[str, torch.Tensor]]) -> Dict[str, float]:
        self.optimizer.zero_grad(set_to_none=True)

        losses_reduced = self.forward_backward(
            forward_step_func=self.forward_step,
            data_iterator=data_iter,
            model=self.model,
            num_microbatches=self.num_microbatches,
            seq_length=self.seq_length,
            micro_batch_size=self.micro_batch_size,
            decoder_seq_length=self.decoder_seq_length,
            forward_only=self.forward_only,
        )

        if not self.forward_only:
            self.optimizer.step()

        return losses_reduced

    def fit(self, train_dataloader: Iterable, steps: int) -> None:
        data_iter = iter(train_dataloader)
        for step in range(1, steps + 1):
            losses = self.training_step(data_iter)
            if step % self.log_interval == 0 and (not torch.distributed.is_initialized() or torch.distributed.get_rank() == 0):
                print(f"[step {step}] losses: {losses}")



def main():
    DistributedEnvironment.initialize(
        tensor_model_parallel_size=1,
        pipeline_model_parallel_size=1,
        expert_model_parallel_size=1,
    )
    model_parallel_cuda_manual_seed(123)

    # Model
    model = GPT20BStyleFactory.build()

    # Data
    dm = GPTMockDataModule(batch_size=1, shuffle=True)
    dm.setup()

    # Optimizer
    optimizer = Adam(
        model.parameters(),
        lr=2e-4,
        betas=(0.9, 0.95),
        eps=1e-8,
        weight_decay=0.1,
    )

    # Trainer
    trainer = MegatronTrainer(
        model=model,
        optimizer=optimizer,
        num_microbatches=1,
        micro_batch_size=2,
        seq_length=_SEQUENCE_LENGTH,
        forward_only=False,
        log_interval=1,
    )

    # Train a few steps (sanity check)
    trainer.fit(dm.train_dataloader(), steps=5)

    ckpt_dir = os.path.join(os.getcwd(), "ckpt")
    CheckpointManager.save(model, ckpt_dir)


    model = CheckpointManager.load(model, ckpt_dir)
    model.to(DistributedEnvironment.device())
    if not torch.distributed.is_initialized() or torch.distributed.get_rank() == 0:
        print("Successfully reloaded checkpoint.")


if __name__ == "__main__":
    main()


Overwriting gpt_oss_trainer.py


In [ ]:
! chmod +x run_trainer.sh
! ./run_trainer.sh